Neste exemplo são apresentados a criação e o treinamento de uma rede neural para análise de sentimanto utilizando Transformer. O objetivo é classificar revisões de filmes do IMDb como positivas ou negativas.

Esta implementação não possui um decoder explícito, pois isso não é necessário para a tarefa de classificação de texto (onde a saída é uma classe e não uma sequência de elementos como no problema de tradução automática ou geração de texto).

##**1 - Importando bibliotecas e módulos**

In [ ]:
import keras
from keras import ops, layers, utils, datasets, models


##**2 - Carregando o dataset e separando os dados**

In [ ]:
max_features = 20000
(X_train, Y_train), (X_val, Y_val) = datasets.imdb.load_data(number_words=max_features)
maxlen = 100
X_train = utils.pad_sequences(X_train, maxlen = maxlen)
X_val = utils.pad_sequences(X_val, maxlen=maxlen)

##**3 - Tratando as Entradas**

In [ ]:
inputs = layers.Input(shape=(maxlen,))
emb_dims = 50
token_emb = layers.Embedding(input_dim=max_features, output_dim=emb_dims)(inputs)
positions = ops.arange(start=0, stop=maxlen, step=1)
pos_emb = layers.Embedding(input_dim=maxlen, output_dim=emb_dims)(positions)
x = token_emb + pos_emb

##**4 - Camada de Auto-Atenção (Self-Attention)**

In [ ]:
num_heads = 5
atencao = layers.MultiHeadAttention(num_heads = num_heads, key_dim = emb_dims)(x,x)
atencao_output = layers.Dropout(0.1)(atencao)
saida1 = layers.LayerNormalization(epsilon=1e-6)(x + atencao_output)

##**5 - Camada Feedforward**

In [ ]:
num_neuronios = 32
feed_output = layers.Dense(num_neuronios, activation = 'relu')(saida1)
feed_output = layers.Dense(emb_dims)(feed_output)
feed_output = layers.Dropout(0.1)(feed_output)
x = layers.LayerNormalization(epsilon=1e-6)(saida1 + feed_output)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation='relu')(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation = 'softmax')(x)

##**6 - Criando e compilando o modelo**

In [ ]:
model = models.Model(inputs = inputs, outputs = outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=32, epochs=2, validation_data = (X_val, Y_val))

Epoch 1/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.7332 - loss: 0.5071 - val_accuracy: 0.8497 - val_loss: 0.3426
Epoch 2/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9068 - loss: 0.2349 - val_accuracy: 0.8429 - val_loss: 0.4119


##**7 - Testando o Modelo**

In [ ]:
indice_da_palavra = datasets.imdb.get_word_index()

def text_to_sequence(text):
  palavras = text.lower().split()
  sequencia = []
  for palavra in palavras:
    if palavra in indice_da_palavra and indice_da_palavra[palavra] < max_features:
      sequencia.append(indice_da_palavra[palavra]+3)
  return utils.pad_sequences([sequencia], maxlen=maxlen)

def classify_review(review):
  sequencia = text_to_sequence(resenha)
  prediction = model.predict(sequencia)
  print(f' Predição da rede: {prediction[0][0]:.5f}')
  return 'Positiva' if prediction[0][0]> 0.5 else 'Negativa'

resenha = 'This movie is very bad and boring. I consider it horrible.'
classification = classify_review(resenha)
print(f'A resenha é classificada como: {classification}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
 Predição da rede: 0.97995
A resenha é classificada como: Positiva
